In [1]:
from dataclasses import dataclass
import re


@dataclass  # This decorator helps save time from initializing constructor
class Flash_card:
    """This contains a question, its demonstration, possible choices, and correct answers out of the choices"""

    question: str
    choices: list
    correct_choices: list
    demo: str = None

    # Shuffle all choices
    def shuffle_choices(self):
        np.random.shuffle(self.choices)

In [2]:
from termcolor import colored
import numpy as np
import roman


class Folder:
    """This contains multiple flash cards"""

    my_flash = []
    correct: int = 0

    def total_score(self):
        """This prints the percentage of correct answers"""
        try:
            print(f"You get {self.correct * 100 / len(self.my_flash):.2f}%")
        except:
            print("You have to have at least one question")

    def learn(self):
        """The user chooses correct answer(s) by typing a, b, c, etc. (lower case) and hit enter"""
        # Shuffle order of questions
        np.random.shuffle(self.my_flash)
        for i, flash in enumerate(self.my_flash, 1):
            print(colored(f"{roman.toRoman(i)}. {flash.question[0].upper() + flash.question[1:]}", "blue"))  # Print question
            if flash.demo:
                print(flash.demo.strip())  # Print question demonstration
            # Shuffle choices
            flash.shuffle_choices()
            for index, c in enumerate(flash.choices):
                print(
                    colored(f"{index + 1}. {c[0].upper() + c[1:]}", "magenta")
                )  # Print possible choices
            answers = []
            if len(flash.correct_choices) > 1:
                answer = input(
                    "Choose multiple answers"
                )
                while any(int(a) > len(flash.choices) for a in answer):
                    answer = input('You have an invalid answer, please choose again')
                for a in list(answer):
                    answers.append(flash.choices[int(a) - 1])
            else:
                ans = int(input("Choose one correct answer"))
                while ans > len(flash.choices):
                    ans = int(input('You have an invalid answer, please choose again'))
                answers.append(flash.choices[ans - 1])
            if sorted(answers) == sorted(flash.correct_choices):  # Correct answer(s)
                print("Correct!")
                self.correct += 1
            else:  # Incorrect answer(s)
                print(colored(f"Wrong! Correct choice should be: ", "yellow"), end="")
                print(colored("\n".join(flash.correct_choices), "red"))
            print()
        self.total_score()

In [3]:
def read_a_file(file_path: str, general_choice_pattern: str, correct_choice_pattern: str) -> Folder:
    """This reads a text file and returns a Folder instance"""
    doc = Folder()
    with open(file_path, "r", encoding='utf8') as f:
        q, demo, correct_choices, choices = "", "", [], []
        for line in f:
            correct = re.match(correct_choice_pattern, line)
            regular = re.match(general_choice_pattern, line)
            # print(f'line: {line}')
            # Read multiple-lined questions
            if not regular:
                q += line
                # print(f'not regular, q: {q}')
            else:
                # print(f'regular, neat q: {q}')
                if correct:  # Get correct choices
                    correct_choice = line[correct.end():].strip()
                    correct_choices.append(correct_choice)
                    # Any way, the regular choice is added
                    choices.append(correct_choice)
                    # print(f'correct_choice: {correct_choice}')
                else:  # Get incorrect choices
                    regular_choice = line[regular.end():].strip()
                    # print(f'incorrect_choice: {regular_choice}')
                    choices.append(regular_choice)
            if line in ['\n', '\r\n']:
                # Extract the question
                # print(f'q before split: {q}')
                q = q.split(' ', 1)[-1]
                # print(f'q after split: {q}')
                q = q.replace('\n', '').strip()
                # print(f'q after replace: {q}')
                a_flash_card = Flash_card(
                    question=q,
                    choices=choices,
                    correct_choices=correct_choices,
                )
                if a_flash_card not in doc.my_flash:
                    doc.my_flash.append(a_flash_card)
                # print(f'question: {q}')
                # print(f'answers: {choices}')
                # print(f'correct answer: {correct_choices}')
                # print()
                q, demo, correct_choices, choices = "", "", [], []
    return doc

In [4]:
def create_a_nicer_file(doc: Folder, filename: str):
    """
    This is to create a nicer Q&A file name from doc a Folder instance
    """
    with open(str(filename), "w", encoding='utf8') as f:
        for i, flash in enumerate(doc.my_flash, 1):
            f.write(f"####Q{i}. {flash.question}\n")  # Write question
            if flash.demo:  # Write demo if it exists
                f.write(flash.demo)
            for choice in flash.choices:
                if choice in flash.correct_choices:
                    f.write(f"- [x] {choice}\n")
                else:
                    f.write(f"- [ ] {choice}\n")
            f.write("\n")

In [5]:
file_name = "5215.txt"
doc = read_a_file(file_name, general_choice_pattern=r'^\s*-\s*\[\s*\w*\s*\]\s*',
                  correct_choice_pattern=r'^\s*-\s*\[\s*\w\s*\]\s*')
create_a_nicer_file(doc, file_name.split(".", 1)[0] + ".txt")
doc.learn()

I. Reward prediction error is
1. The expected reward value - the received reward value
2. The total future expected reward minus the total future actual reward (with temporal discounting)
3. The received reward value - the expected reward value
4. The total future actual reward minus the total future expected reward (with temporal discounting)


Choose one correct answer 3


Correct!

II. If you are teams #2 and #3 in a competition and you want to merge your teams to beat #1, you will more likely be using bagging rather than boosting to win.
1. False
2. True


Choose one correct answer 1


Correct!

III. Backpropagation is the ability of deep learning neural networks, like google's Deep Dream, to propagate signals from higher level neurons down to low-level neurons - allowing us in some cases to see the "hallucinated" images.
1. False
2. True


...

Correct!

XLIX. Check all the ways of initializing K-means
1. Initialize all centroids at the origin - e.g. (0,0)
2. Randomly choose samples as the initial centroids
3. Randomly assign all samples to one of K classes
4. Initialize all centroids at the mean of all the samples


Choose multiple answers 23


Correct!

L. Which is an example of "explaining away"? That is, how a shared child node can indicate a dependency among parent nodes only when observed.
1. When you see someone smoking, you know their odds of having a positive lung X-ray for cancer are higher.
2. Seeing someone with lung cancer is smoking makes you less likely to assume the cancer is from high levels of pollution.
3. A positive result on a lung X-ray indicates a high probability that someone has lung cancer


Choose one correct answer 2


Correct!

You get 96.00%
